In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [21]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/Tourism_in_Bangladesh")
loader

In [22]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Tourism_in_Bangladesh', 'title': 'Tourism in Bangladesh - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nTourism in Bangladesh - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n

In [23]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [24]:
documents

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Tourism_in_Bangladesh', 'title': 'Tourism in Bangladesh - Wikipedia', 'language': 'en'}, page_content="Tourism in Bangladesh - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\

In [25]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [26]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [27]:
vectorstoredb

In [31]:
## Query From a vector db
query="Ratargul Swamp Forest"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'Madhobpur Lake[edit]\nMain article: Madhobpur Lake\nMadhobpur Lake is a lake of Srimangal in Maulvi Bazar District of Bangladesh.[17] It is one of the popular tourist spot in Bangladesh.[18][19]\n\nRatargul Swamp Forest[edit]\nMain article: Ratargul Swamp ForestRatargul Swamp Forest is a freshwater swamp forest located in Gowainghat, Sylhet. It is the only swamp forest located in Bangladesh and one of the few freshwater swamp forest in the world. The forest is naturally conserved under the Department of Forestry, Govt. of Bangladesh.\nRatargul Swamp forest Entrance.\nThe evergreen forest[20] is situated by the river Goain and linked with the channel Chengir Khal. Most of the trees growing here are the Millettia pinnata (করচ গাছ "Koroch tree"). The forest is submerged under 20–30 feet water in the rainy season. For the rest of the year, the water level is about 10 feet deep.[21][22][23]'

In [32]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [33]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f83b80beb30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f83b80bc700>, root_client=<openai.OpenAI object at 0x7f83b80c8c70>, root_async_client=<openai.AsyncOpenAI object at 0x7f83b80befb0>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [34]:
### Input--->Retriever--->vectorstoredb
from langchain.chains import create_retrieval_chain

retriever=vectorstoredb.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f83b8028af0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'))])
            | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f83b80beb30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f83b80bc700>, 

In [37]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"tourism in dhaka?"})
response['answer']

'What is the main role of the Tourist Police unit in Bangladesh?\n\nThe Tourist Police unit in Bangladesh is formed to protect local and foreign tourists better and look after the nature and wildlife in the tourist spots.'

In [38]:
response

{'input': 'tourism in dhaka?',
 'context': [Document(metadata={'source': 'https://en.wikipedia.org/wiki/Tourism_in_Bangladesh', 'title': 'Tourism in Bangladesh - Wikipedia', 'language': 'en'}, page_content='Tourist attractions in Dhaka[edit]\nLalbagh Fort\nDhakeshwari Temple\nBaitul Mukarram\nAhsan Manzil\nShaheed Minar\nLiberation War Museum\nRickshaw art and rickshaw-riding\nJatiyo Sangsad Bhaban\nBashundhara City\nBangladesh National Museum\nNikli Haor\nNorth Bengal (Rajshahi and Rangpur Division)[edit]\nShiva Temple in Puthia, Rajshahi\nTajhat Palace at Rangpur, Bangladesh\nSomapura Mahavihara[edit]\nMain article: Somapura Mahavihara\nSomapura Mahavihara is a World Heritage Site.\nSomapura Mahavihara in Paharpur, Badalgachhi Upazila, Naogaon District (25°1\'51.83"N, 88°58\'37.15"E) is among the best known Buddhist viharas in the Indian Subcontinent and is one of the most important archaeological sites in the country. It was designated a UNESCO World Heritage Site in 1985.'),
  Docu

In [39]:
response['context']

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Tourism_in_Bangladesh', 'title': 'Tourism in Bangladesh - Wikipedia', 'language': 'en'}, page_content='Tourist attractions in Dhaka[edit]\nLalbagh Fort\nDhakeshwari Temple\nBaitul Mukarram\nAhsan Manzil\nShaheed Minar\nLiberation War Museum\nRickshaw art and rickshaw-riding\nJatiyo Sangsad Bhaban\nBashundhara City\nBangladesh National Museum\nNikli Haor\nNorth Bengal (Rajshahi and Rangpur Division)[edit]\nShiva Temple in Puthia, Rajshahi\nTajhat Palace at Rangpur, Bangladesh\nSomapura Mahavihara[edit]\nMain article: Somapura Mahavihara\nSomapura Mahavihara is a World Heritage Site.\nSomapura Mahavihara in Paharpur, Badalgachhi Upazila, Naogaon District (25°1\'51.83"N, 88°58\'37.15"E) is among the best known Buddhist viharas in the Indian Subcontinent and is one of the most important archaeological sites in the country. It was designated a UNESCO World Heritage Site in 1985.'),
 Document(metadata={'source': 'https://en.wikiped